In [1]:
#IMPORTS

import numpy as np
import matplotlib.pyplot as plt
import pygame
import random

from pygame.locals import (
    RLEACCEL,
    K_UP,
    K_DOWN,
    K_LEFT,
    K_RIGHT,
    K_ESCAPE,
    KEYDOWN,
    QUIT,
)


pygame 2.1.2 (SDL 2.0.16, Python 3.8.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
#CLASS DEFINITIONS

class Drone(pygame.sprite.Sprite):
    def __init__(self, xi, yi, zi, mode):
        self.x = xi
        self.y = yi
        self.z = zi
        self.mode = mode
        super(Drone, self).__init__()
        self.surf = pygame.Surface((10, 10))
        self.surf.fill((0, 191, 255))
        self.rect = self.surf.get_rect(
            center=(xi,yi,)
        )

        self.direction = np.pi / 2
        #self.direction = random.randint(0,43) / 7
        self.force = 10

        self.l = self.rect.left
        self.t = self.rect.top
        self.prevl = self.l
        self.prevt = self.t

    def update(self,SCREEN_WIDTH, SCREEN_HEIGHT):

        self.prevl = self.l
        self.prevt = self.t


        if self.mode==0:
            self.direction=0
            self.force=0
        elif self.mode==1:
            self.direction = random.randint(0,43) / 7
            self.force = 10
        elif self.mode==2:
            self.direction = 0
            self.force = 10

        if self.force > 0:
            dx = round(self.force * np.cos(self.direction))
            dy = -1*round(self.force * np.sin(self.direction))
            self.rect.move_ip(dx,dy)

        if self.rect.left < 0:
            self.rect.left = 0
        if self.rect.right > SCREEN_WIDTH:
            self.rect.right = SCREEN_WIDTH
        if self.rect.top <= 0:
            self.rect.top = 0
        if self.rect.bottom >= SCREEN_HEIGHT:
            self.rect.bottom = SCREEN_HEIGHT

        self.l = self.rect.left
        self.t = self.rect.top

class Goal(pygame.sprite.Sprite):
    def __init__(self, x, y, z):
        super(Goal, self).__init__()
        self.surf = pygame.Surface((10, 10))
        self.surf.fill((57,255,20))
        self.rect = self.surf.get_rect(
            center=(x,y,)
        )

class Obstacle(pygame.sprite.Sprite):
    def __init__(self,x,y,z,xdim,ydim,zdim):
        super(Obstacle, self).__init__()
        self.surf = pygame.Surface((xdim,ydim))
        self.surf.fill((255,0,0))
        self.rect = self.surf.get_rect(
            center=(x,y,)
        )

class Scanner(pygame.sprite.Sprite):
    def __init__(self,left,top,xdim,ydim,dir):
        super(Scanner, self).__init__()
        self.surf = pygame.Surface((xdim,ydim))
        self.surf.fill((255,255,255))
        self.rect = self.surf.get_rect()
        self.rect.left = left
        self.rect.top = top
        self.dir = dir

    def update(self):
        if self.dir == "r":
            self.rect.move_ip(1,0)
        elif self.dir == "l":
            self.rect.move_ip(-1,0)
        elif self.dir == "u":
            self.rect.move_ip(0,-1)
        elif self.dir == "d":
            self.rect.move_ip(0,1)
        elif self.dir == "ur":
            self.rect.move_ip(1,-1)
        elif self.dir == "ul":
            self.rect.move_ip(-1,-1)
        elif self.dir == "dl":
            self.rect.move_ip(-1,1)
        elif self.dir == "dr":
            self.rect.move_ip(1,1)

        

In [33]:
#OBSERVE() DEFINITION

def observe(entity,spritesgroup,dist,scansgroup,ogroup,ggroup,dgroup):
    l=entity.rect.left
    lx=l-1
    r=entity.rect.right
    t=entity.rect.top
    tx=t-1
    b=entity.rect.bottom

    txx = t-10
    lxx = l-10


    results = np.array([[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]])

    rscan = Scanner(r,t,1,10,"r")
    uscan = Scanner(l,tx,10,1,"u")
    lscan = Scanner(lx,t,1,10,"l")
    dscan = Scanner(l,b,10,1,"d")
    urscan = Scanner(r,txx,10,10,"ur")
    ulscan = Scanner(lxx,txx,10,10,"ul")
    dlscan = Scanner(lxx,b,10,10,"dl")
    drscan = Scanner(r,b,10,10,"dr")
    scansgroup.add(rscan,uscan,lscan,dscan,urscan,ulscan,dlscan,drscan)

    for i in range(dist):
        if pygame.sprite.spritecollideany(rscan,spritesgroup):
            if pygame.sprite.spritecollideany(rscan,ogroup):
                results[0] = [1,i]
            elif pygame.sprite.spritecollideany(rscan,ggroup):
                results[0] = [2,i]
            elif pygame.sprite.spritecollideany(rscan,dgroup):
                results[0] = [3,i]
            break
        rscan.update()

    for i in range(dist):
        if pygame.sprite.spritecollideany(uscan,spritesgroup):
            if pygame.sprite.spritecollideany(uscan,ogroup):
                results[1] = [1,i]
            elif pygame.sprite.spritecollideany(uscan,ggroup):
                results[1] = [2,i]
            elif pygame.sprite.spritecollideany(uscan,dgroup):
                results[1] = [3,i]
            break
        uscan.update()

    for i in range(dist):
        if pygame.sprite.spritecollideany(lscan,spritesgroup):
            if pygame.sprite.spritecollideany(lscan,ogroup):
                results[2] = [1,i]
            elif pygame.sprite.spritecollideany(lscan,ggroup):
                results[2] = [2,i]
            elif pygame.sprite.spritecollideany(lscan,dgroup):
                results[2] = [3,i]
            break
        lscan.update()

    for i in range(dist):
        if pygame.sprite.spritecollideany(dscan,spritesgroup):
            if pygame.sprite.spritecollideany(dscan,ogroup):
                results[3] = [1,i]
            elif pygame.sprite.spritecollideany(dscan,ggroup):
                results[3] = [2,i]
            elif pygame.sprite.spritecollideany(dscan,dgroup):
                results[3] = [3,i]
            break
        dscan.update()

    for i in range(dist):
        if pygame.sprite.spritecollideany(urscan,spritesgroup):
            if pygame.sprite.spritecollideany(urscan,ogroup):
                results[4] = [1,i+19]
            elif pygame.sprite.spritecollideany(urscan,ggroup):
                results[4] = [2,i+19]
            elif pygame.sprite.spritecollideany(urscan,dgroup):
                results[4] = [3,i+19]
            break
        urscan.update()

    for i in range(dist):
        if pygame.sprite.spritecollideany(ulscan,spritesgroup):
            if pygame.sprite.spritecollideany(ulscan,ogroup):
                results[5] = [1,i+19]
            elif pygame.sprite.spritecollideany(ulscan,ggroup):
                results[5] = [2,i+19]
            elif pygame.sprite.spritecollideany(ulscan,dgroup):
                results[5] = [3,i+19]
            break
        ulscan.update()

    for i in range(dist):
        if pygame.sprite.spritecollideany(dlscan,spritesgroup):
            if pygame.sprite.spritecollideany(dlscan,ogroup):
                results[6] = [1,i+19]
            elif pygame.sprite.spritecollideany(dlscan,ggroup):
                results[6] = [2,i+19]
            elif pygame.sprite.spritecollideany(dlscan,dgroup):
                results[6] = [3,i+19]
            break
        dlscan.update()

    for i in range(dist):
        if pygame.sprite.spritecollideany(drscan,spritesgroup):
            if pygame.sprite.spritecollideany(drscan,ogroup):
                results[7] = [1,i+19]
            elif pygame.sprite.spritecollideany(drscan,ggroup):
                results[7] = [2,i+19]
            elif pygame.sprite.spritecollideany(drscan,dgroup):
                results[7] = [3,i+19]
            break
        drscan.update()


    for entity in scansgroup:
        entity.kill()

    return results

In [36]:
#ONESIM() DEFINITION

def onesim(maxlen):
    pygame.init()

    clock = pygame.time.Clock()

    #Sets the height and width of the environment
    SCREEN_WIDTH = 800
    SCREEN_HEIGHT = 600

    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

    # output array is num_frames*num_drones*num_coords
    posarr = np.zeros((maxlen,6,3))

    allsprites = pygame.sprite.Group()

    # 6 drones are currently hardcoded in with starting positions
    d1 = Drone(50,200,0,1)
    d2 = Drone(100,250,0,1)
    d3 = Drone(60,300,0,1)
    d4 = Drone(100,300,0,2)
    d5 = Drone(90,400,0,2)
    d6 = Drone(100,500,0,2)

    drones = pygame.sprite.Group()
    drones.add(d1,d2,d3,d4,d5,d6)
    allsprites.add(d1,d2,d3,d4,d5,d6)

    dronelist = [d1,d2,d3,d4,d5,d6]

    # A green 'goal' is randomly placed somehwere on the right side of the environment
    ygoal = np.random.randint(150,450)
    endgoal = Goal(750,ygoal,0)

    goals = pygame.sprite.Group()
    goals.add(endgoal)
    allsprites.add(endgoal)

    goal_l = endgoal.rect.left
    goal_t = endgoal.rect.top

    # Two obstacles randomly placed in the middle of the environment
    yo1 = np.random.randint(150,450)
    o1 = Obstacle(350,yo1,0,20,300,0)

    yo2 = np.random.randint(150,450)
    o2 = Obstacle(550,yo2,0,20,300,0)

    obstacles = pygame.sprite.Group()
    obstacles.add(o1,o2)
    allsprites.add(o1,o2)

    # If running is ever false, the run terminates
    running = True
    counter = 0

    scans = pygame.sprite.Group()

    while running:
        # Can force exit the simulation using ESC or by closing the window
        for event in pygame.event.get():
            if event.type == KEYDOWN:
                if event.key == K_ESCAPE:
                    running = False

            elif event.type == QUIT:
                running = False

        for entity in drones:
            x = observe(entity,allsprites,100,scans,obstacles,goals,drones)
            print(x)


        # Drones update, according to the update function defined in Drone class
        #drones.update(SCREEN_WIDTH,SCREEN_HEIGHT)
        for entity in drones:
            prevl = entity.rect.left
            prevt = entity.rect.top
            prevdist = np.sqrt((goal_l-prevl)**2 + (goal_t-prevt)**2)

            entity.update(SCREEN_WIDTH,SCREEN_HEIGHT)
            
            nowl = entity.rect.left
            nowt = entity.rect.top
            nowdist = np.sqrt((goal_l-nowl)**2 + (goal_t-nowt)**2)

            gamma = 0.999

            reward = gamma**counter * (prevdist-nowdist)

            #print(goal_l,goal_t,prevl,prevt,prevdist,nowl,nowt,nowdist,reward)

        #print()



        screen.fill((0,0,0))

        for entity in allsprites:
            screen.blit(entity.surf, entity.rect)
            
        for entity in drones:
            # Drone is killed if it collides with an obstacle
            if pygame.sprite.spritecollideany(entity, obstacles):
                entity.kill()

            # Drone is killed if it collides with the goal (success!)
            if pygame.sprite.spritecollideany(entity,goals):
                entity.kill()

            # Both drones are killed if they collide with each other
            for entity2 in drones:
                if entity != entity2 and pygame.sprite.collide_rect(entity, entity2):
                    entity.kill()
                    entity2.kill()
    
        pygame.display.flip()

        dead = 0

        # Store results in posarr
        for i in range(len(dronelist)):
            entity = dronelist[i]
            if entity.alive():
                posarr[counter,i] = [entity.rect.left,entity.rect.top,0]
            else:
                dead += 1
                posarr[counter,i] = [-1,-1,-1]

        # Run ends early if all drones are dead
        if dead == 6:
            running = False

        # Run ends if max length has been reached
        counter += 1
        if counter >= maxlen:
            running = False

        # I'm not sure if this is helping the code run more smoothly or not
        clock.tick(60)

    pygame.quit()
    
    return posarr

In [19]:
#MULTISIM() DEFINITION

def multisim(numsims,maxlen):
    multisimlist = []
    for i in range(numsims):
        x=onesim(maxlen)
        multisimlist.append(x)
    multisimlist=np.array(multisimlist)

    return multisimlist

In [38]:
x=onesim(100)

[[ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 3 50]]
[[ 0  0]
 [ 0  0]
 [ 0  0]
 [ 3 40]
 [ 0  0]
 [ 3 50]
 [ 3 50]
 [ 0  0]]
[[ 3 30]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 3 50]
 [ 0  0]
 [ 0  0]
 [ 0  0]]
[[ 0  0]
 [ 3 40]
 [ 3 30]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]]
[[ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 3 51]]
[[ 0  0]
 [ 0  0]
 [ 0  0]
 [ 3 30]
 [ 0  0]
 [ 3 51]
 [ 3 50]
 [ 0  0]]
[[ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 3 50]
 [ 0  0]
 [ 0  0]
 [ 0  0]]
[[ 0  0]
 [ 3 30]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]]
[[ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 3 66]]
[[ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 3 66]
 [ 3 51]
 [ 3 31]]
[[ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 3 5

In [98]:
print(x[99])

[[ 96. 212.   0.]
 [ -1.  -1.  -1.]
 [ -1.  -1.  -1.]
 [ -1.  -1.  -1.]
 [ -1.  -1.  -1.]
 [ -1.  -1.  -1.]]


In [7]:
z=multisim(3,300)

[[ 0  0]
 [ 0  0]
 [ 3 29]
 [ 0  0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[ 3 29]
 [ 0  0]
 [ 0  0]
 [ 0  0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[ 0  0]
 [ 0  0]
 [ 3 26]
 [ 3 97]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[ 0  0]
 [ 0  0]
 [ 0  0]
 [ 3 35]]
[[ 3 26]
 [ 3 35]
 [ 0  0]
 [ 0  0]]
[[ 0  0]
 [ 3 97]
 [ 0  0]
 [ 0  0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[ 0  0]
 [ 0  0]
 [ 3 20]
 [ 3 93]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[ 3 20]
 [ 0  0]
 [ 0  0]
 [ 0  0]]
[[ 0  0]
 [ 3 93]
 [ 0  0]
 [ 0  0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [3 6]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[3 6]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]


In [40]:
np.shape(z)

(3, 100, 6, 3)